This is model 1.0.0.29 trained on dataset 8 classifying to all five categories. The model was actually trained on a Google Cloud instance with GPU as it was far too computationally expensive to train on my laptop. When training the metrics were saved to disk and those are used for the plots below. Metrics were also logged to TensorBoard. Those logs as well as the resulting model are included in the repository.

In [1]:
import numpy as np
import os
import wget
import tensorflow as tf
from training_utils import download_file, get_batches, read_and_decode_single_example, load_validation_data, \
    download_data, evaluate_model, get_training_data, load_weights, flatten
import matplotlib.pyplot as plt
%matplotlib inline
#import argparse
from tensorboard import summary as summary_lib

C:\Users\eric\Anaconda2\envs\exts-aml2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# arguments
epochs = 35
dataset = 9
init_model = None
how = "normal"
action = "train"
threshold = 0.5

In [3]:
# download the data
#download_data(what=dataset)
batch_size = 32
train_files, total_records = get_training_data(what=dataset)

In [4]:
## Hyperparameters
epsilon = 1e-8

# learning rate
epochs_per_decay = 10
starting_rate = 0.001
decay_factor = 0.80
staircase = True

# learning rate decay variables
steps_per_epoch = int(total_records / batch_size)
print("Steps per epoch:", steps_per_epoch)

# lambdas
lamC = 0.00001
lamF = 0.00250

mu = 104

# use dropout
dropout = True
fcdropout_rate = 0.5
convdropout_rate = 0.001
pooldropout_rate = 0.1

if how == "label":
    num_classes = 5
elif how == "normal":
    num_classes = 2
elif how == "mass":
    num_classes = 3
elif how == "benign":
    num_classes = 3

print("Number of classes:", num_classes)

config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.6

Steps per epoch: 1366
Number of classes: 2


In [5]:
model_name = "model_s1.0.0.29l.14"
## Change Log
# 0.0.0.4 - increase pool3 to 3x3 with stride 3
# 0.0.0.6 - reduce pool 3 stride back to 2
# 0.0.0.7 - reduce lambda for l2 reg
# 0.0.0.8 - increase conv1 to 7x7 stride 2
# 0.0.0.9 - disable per image normalization
# 0.0.0.10 - commented out batch norm in conv layers, added conv4 and changed stride of convs to 1, increased FC lambda
# 0.0.0.11 - turn dropout for conv layers on
# 0.0.0.12 - added batch norm after pooling layers, increase pool dropout, decrease conv dropout, added extra conv layer to reduce data dimensionality
# 0.0.0.13 - added precision and f1 summaries
# 0.0.0.14 - fixing batch normalization, I don't think it's going to work after each pool
# 0.0.0.15 - reduced xentropy weighting term
# 0.0.0.17 - replaced initial 5x5 conv layers with 3 3x3 layers
# 0.0.0.18 - changed stride of first conv to 2 from 1
# 0.0.0.19 - doubled units in two fc layers
# 0.0.0.20 - lowered learning rate, put a batch norm back in
# 0.0.0.21 - put all batch norms back in
# 0.0.0.22 - increased lambdaC, removed dropout from conv layers
# 1.0.0.23 - added extra conv layers
# 1.0.0.27 - updates to training code and metrics
# 1.0.0.28 - using weighted x-entropy to improve recall
# 1.0.0.29 - updated code to work training to classify for multiple classes
# 1.0.0.29f - putting weighted x-entropy back

graph = tf.Graph()
with graph.as_default():
    training = tf.placeholder(dtype=tf.bool, name="is_training")
    is_testing = tf.placeholder(dtype=bool, shape=(), name="is_testing")

    # create global step for decaying learning rate
    global_step = tf.Variable(0, trainable=False)

    learning_rate = tf.train.exponential_decay(starting_rate,
                                               global_step,
                                               steps_per_epoch * epochs_per_decay,
                                               decay_factor,
                                               staircase=staircase)

    with tf.name_scope('inputs') as scope:
        image, label = read_and_decode_single_example(train_files, label_type=how, normalize=False)

        X_def, y_def = tf.train.shuffle_batch([image, label], batch_size=batch_size, capacity=2000,
                                              min_after_dequeue=1000)

        # Placeholders
        X = tf.placeholder_with_default(X_def, shape=[None, 299, 299, 1])
        y = tf.placeholder_with_default(y_def, shape=[None])
        
        X_adj = tf.image.adjust_contrast(X, 2.0)
        X_adj = tf.cast(X_adj, dtype=tf.float32)

        # center the pixel data
        mu = tf.constant(mu, name="pixel_mean", dtype=tf.float32)
        X_adj = tf.subtract(X_adj, mu, name="centered_input")

        # scale the data
        X_adj = tf.divide(X_adj, 255.0)
        
    # Convolutional layer 1
    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(
            X_adj,  # Input data
            filters=32,
            kernel_size=(3, 3),
            strides=(2, 2),
            padding='SAME',
            activation=None,
            kernel_initializer=tf.truncated_normal_initializer(stddev=5e-2, seed=100),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=lamC),
            name='conv1'
        )

        conv1 = tf.layers.batch_normalization(
            conv1,
            axis=-1,
            momentum=0.99,
            epsilon=epsilon,
            center=True,
            scale=True,
            beta_initializer=tf.zeros_initializer(),
            gamma_initializer=tf.ones_initializer(),
            moving_mean_initializer=tf.zeros_initializer(),
            moving_variance_initializer=tf.ones_initializer(),
            training=training,
            name='bn1'
        )

        # apply relu
        conv1 = tf.nn.relu(conv1, name='relu1')
        
    print("Graph created...")

Instructions for updating:
Use the retry module or similar alternatives.
Graph created...


In [6]:
## CONFIGURE OPTIONS
# if we are initializing the weights from a pre-trained model make sure it exists
if init_model is not None:
    if os.path.exists(os.path.join("model", init_model + '.ckpt.index')):
        init = False
    else:
        init = True
# otherwise if this model exists let's continue training it rather than reinitializing it
else:
    if os.path.exists(os.path.join("model", model_name + '.ckpt.index')):
        init = False
    else:
        init = True

meta_data_every = 1
log_to_tensorboard = True
print_every = 5  # how often to print metrics
checkpoint_every = 1  # how often to save model in epochs
use_gpu = False  # whether or not to use the GPU
print_metrics = True  # whether to print or plot metrics, if False a plot will be created and updated every epoch

config = tf.ConfigProto(device_count = {'GPU': 0})

In [7]:
init = True
## train the model
with tf.Session(graph=graph, config=config) as sess:
    if log_to_tensorboard:
        train_writer = tf.summary.FileWriter('./logs/tr_' + model_name, sess.graph)
        test_writer = tf.summary.FileWriter('./logs/te_' + model_name)

    if not print_metrics:
        # create a plot to be updated as model is trained
        f, ax = plt.subplots(1, 4, figsize=(24, 5))

    # create the saver
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    print("Initializing model...")

    # if we are training the model
    if action == "train":
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        print("Training model", model_name, "...")

        for epoch in range(2):
            sess.run(tf.local_variables_initializer())

            for i in range(steps_per_epoch):
                # create the metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()

            
                image_batch, conv1_batch = sess.run(
                    [X_adj, conv1],
                    feed_dict={
                        training: True,
                    },
                    options=run_options,
                    run_metadata=run_metadata)
                
                image_arr = np.array(image_batch)
                conv1_batch = np.array(conv1_batch)
#                 plt.imshow(image_arr[0].reshape([299,299]))
#                 plt.show()
                print("Train Mean:", np.mean(image_arr))
                print("Train Min:", np.min(image_arr))
                print("Train Max:", np.max(image_arr))
            
                print("Train C1 Mean:", np.mean(conv1_batch))
                print("Train C1 Min:", np.min(conv1_batch))
                print("Train C1 Max:", np.max(conv1_batch))
                break
            
            # initialize the local variables so we have metrics only on the evaluation
            sess.run(tf.local_variables_initializer())

            print("Evaluating model...")
            # load the test data
            X_cv, y_cv = load_validation_data(percentage=1, how=how, which=dataset)

            # evaluate the test data
            for X_batch, y_batch in get_batches(X_cv, y_cv, batch_size, distort=False):
                cv_image_batch, cv_conv1_batch = sess.run(
                    [X_adj, conv1],
                    feed_dict={
                        X: X_batch,
                        y: y_batch,
                        training: False
                    })
                break
            
            cv_image_arr = np.array(cv_image_batch)
            cv_conv1_batch = np.array(cv_conv1_batch)
#             plt.imshow(cv_image_arr[0].reshape([299,299]))
#             plt.show()
            print("CV Mean:", np.mean(cv_image_arr))
            print("CV Min:", np.min(cv_image_arr))
            print("CV Max:", np.max(cv_image_arr))
            
            print("CV C1 Mean:", np.mean(cv_conv1_batch))
            print("CV C1 Min:", np.min(cv_conv1_batch))
            print("CV C1 Max:", np.max(cv_conv1_batch))
            
            # delete the test data to save memory
            del (X_cv)
            del (y_cv)

            print("Done evaluating...")

            break
        # stop the coordinator
        coord.request_stop()

        # Wait for threads to stop
        coord.join(threads)

Initializing model...
Training model model_s1.0.0.29l.14 ...
Train Mean: 0.009218497
Train Min: -0.40784317
Train Max: 0.5921569
Train C1 Mean: 0.3600788
Train C1 Min: 0.0
Train C1 Max: 12.572195
Evaluating model...
CV Mean: 0.008187771
CV Min: -0.40784317
CV Max: 0.5921569
CV C1 Mean: 0.0068575907
CV C1 Min: 0.0
CV C1 Max: 0.16763291
Done evaluating...


In [8]:
train_image = image_arr[0]
cv_image = cv_image_arr[0]

In [9]:
print(np.mean(train_image))
print(np.mean(cv_image))

0.40100044
0.047194194
